<a href="https://colab.research.google.com/github/antonioalbanese/Time-Series-Anomaly-Detection-An-experimental-survey/blob/main/TadGAN/TadGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/arunppsg/TadGAN
%cd TadGAN

Cloning into 'TadGAN'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 55 (delta 9), reused 8 (delta 8), pack-reused 42
Unpacking objects: 100% (55/55), done.
/content/TadGAN


In [2]:
#to have nab dataset
!git clone https://github.com/antonioalbanese/Time-Series-Anomaly-Detection-An-experimental-survey/

Cloning into 'Time-Series-Anomaly-Detection-An-experimental-survey'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 114 (delta 13), reused 92 (delta 12), pack-reused 17
Receiving objects: 100% (114/114), 1.90 MiB | 21.16 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [3]:
##################################
########### FUNCTIONS ############
##################################
import os
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import model
import anomaly_detection

# logging.basicConfig(filename='train.log', level=logging.DEBUG)

class SignalDataset(Dataset):
    def __init__(self, path):
        self.signal_df = pd.read_csv(path)
        self.signal_columns = self.make_signal_list()
        self.make_rolling_signals()

    def make_signal_list(self):
        signal_list = list()
        for i in range(-50, 50):
            signal_list.append('signal'+str(i))
        return signal_list

    def make_rolling_signals(self):
        for i in range(-50, 50):
            self.signal_df['signal'+str(i)] = self.signal_df['signal'].shift(i)
        self.signal_df = self.signal_df.dropna()
        self.signal_df = self.signal_df.reset_index(drop=True)

    def __len__(self):
        return len(self.signal_df)

    def __getitem__(self, idx):
        row = self.signal_df.loc[idx]
        x = row[self.signal_columns].values.astype(float)
        x = torch.from_numpy(x)
        return {'signal':x, 'anomaly':row['anomaly']}

def critic_x_iteration(sample):
    optim_cx.zero_grad()

    x = sample['signal'].view(1, batch_size, signal_shape)
    valid_x = critic_x(x)
    valid_x = torch.squeeze(valid_x)
    critic_score_valid_x = torch.mean(torch.ones(valid_x.shape) * valid_x) #Wasserstein Loss

    #The sampled z are the anomalous points - points deviating from actual distribution of z (obtained through encoding x)
    z = torch.empty(1, batch_size, latent_space_dim).uniform_(0, 1)
    x_ = decoder(z)
    fake_x = critic_x(x_)
    fake_x = torch.squeeze(fake_x)
    critic_score_fake_x = torch.mean(torch.ones(fake_x.shape) * fake_x)  #Wasserstein Loss

    alpha = torch.rand(x.shape)
    ix = Variable(alpha * x + (1 - alpha) * x_) #Random Weighted Average
    ix.requires_grad_(True)
    v_ix = critic_x(ix)
    v_ix.mean().backward()
    gradients = ix.grad
    #Gradient Penalty Loss
    gp_loss = torch.sqrt(torch.sum(torch.square(gradients).view(-1)))

    #Critic has to maximize Cx(Valid X) - Cx(Fake X).
    #Maximizing the above is same as minimizing the negative.
    wl = critic_score_fake_x - critic_score_valid_x
    loss = wl + gp_loss
    loss.backward()
    optim_cx.step()

    return loss

def critic_z_iteration(sample):
    optim_cz.zero_grad()

    x = sample['signal'].view(1, batch_size, signal_shape)
    z = encoder(x)
    valid_z = critic_z(z)
    valid_z = torch.squeeze(valid_z)
    critic_score_valid_z = torch.mean(torch.ones(valid_z.shape) * valid_z)

    z_ = torch.empty(1, batch_size, latent_space_dim).uniform_(0, 1)
    fake_z = critic_z(z_)
    fake_z = torch.squeeze(fake_z)
    critic_score_fake_z = torch.mean(torch.ones(fake_z.shape) * fake_z) #Wasserstein Loss

    wl = critic_score_fake_z - critic_score_valid_z

    alpha = torch.rand(z.shape)
    iz = Variable(alpha * z + (1 - alpha) * z_) #Random Weighted Average
    iz.requires_grad_(True)
    v_iz = critic_z(iz)
    v_iz.mean().backward()
    gradients = iz.grad
    gp_loss = torch.sqrt(torch.sum(torch.square(gradients).view(-1)))

    loss = wl + gp_loss
    loss.backward()
    optim_cz.step()

    return loss

def encoder_iteration(sample):
    optim_enc.zero_grad()

    x = sample['signal'].view(1, batch_size, signal_shape)
    valid_x = critic_x(x)
    valid_x = torch.squeeze(valid_x)
    critic_score_valid_x = torch.mean(torch.ones(valid_x.shape) * valid_x) #Wasserstein Loss

    z = torch.empty(1, batch_size, latent_space_dim).uniform_(0, 1)
    x_ = decoder(z)
    fake_x = critic_x(x_)
    fake_x = torch.squeeze(fake_x)
    critic_score_fake_x = torch.mean(torch.ones(fake_x.shape) * fake_x)

    enc_z = encoder(x)
    gen_x = decoder(enc_z)

    mse = mse_loss(x.float(), gen_x.float())
    loss_enc = mse + critic_score_valid_x - critic_score_fake_x
    loss_enc.backward(retain_graph=True)
    optim_enc.step()

    return loss_enc

def decoder_iteration(sample):
    optim_dec.zero_grad()

    x = sample['signal'].view(1, batch_size, signal_shape)
    z = encoder(x)
    valid_z = critic_z(z)
    valid_z = torch.squeeze(valid_z)
    critic_score_valid_z = torch.mean(torch.ones(valid_z.shape) * valid_z)

    z_ = torch.empty(1, batch_size, latent_space_dim).uniform_(0, 1)
    fake_z = critic_z(z_)
    fake_z = torch.squeeze(fake_z)
    critic_score_fake_z = torch.mean(torch.ones(fake_z.shape) * fake_z)

    enc_z = encoder(x)
    gen_x = decoder(enc_z)

    mse = mse_loss(x.float(), gen_x.float())
    loss_dec = mse + critic_score_valid_z - critic_score_fake_z
    loss_dec.backward(retain_graph=True)
    optim_dec.step()

    return loss_dec


def train(n_epochs=2000):
    # logging.debug('Starting training')
    print('Starting training')
    cx_epoch_loss = list()
    cz_epoch_loss = list()
    encoder_epoch_loss = list()
    decoder_epoch_loss = list()

    for epoch in range(n_epochs):
        print('Epoch {}'.format(epoch))
        n_critics = 5

        cx_nc_loss = list()
        cz_nc_loss = list()

        for i in range(n_critics):
            cx_loss = list()
            cz_loss = list()

            for batch, sample in enumerate(train_loader):
                loss = critic_x_iteration(sample)
                cx_loss.append(loss)

                loss = critic_z_iteration(sample)
                cz_loss.append(loss)

            cx_nc_loss.append(torch.mean(torch.tensor(cx_loss)))
            cz_nc_loss.append(torch.mean(torch.tensor(cz_loss)))

        print('Critic training done in epoch {}'.format(epoch))
        encoder_loss = list()
        decoder_loss = list()

        for batch, sample in enumerate(train_loader):
            enc_loss = encoder_iteration(sample)
            dec_loss = decoder_iteration(sample)
            encoder_loss.append(enc_loss)
            decoder_loss.append(dec_loss)

        cx_epoch_loss.append(torch.mean(torch.tensor(cx_nc_loss)))
        cz_epoch_loss.append(torch.mean(torch.tensor(cz_nc_loss)))
        encoder_epoch_loss.append(torch.mean(torch.tensor(encoder_loss)))
        decoder_epoch_loss.append(torch.mean(torch.tensor(decoder_loss)))
        print('Encoder decoder training done in epoch {}'.format(epoch))
        print('critic x loss {:.3f} critic z loss {:.3f} \nencoder loss {:.3f} decoder loss {:.3f}\n'.format(cx_epoch_loss[-1], cz_epoch_loss[-1], encoder_epoch_loss[-1], decoder_epoch_loss[-1]))

        if epoch % 10 == 0:
            torch.save(encoder.state_dict(), encoder.encoder_path)
            torch.save(decoder.state_dict(), decoder.decoder_path)
            torch.save(critic_x.state_dict(), critic_x.critic_x_path)
            torch.save(critic_z.state_dict(), critic_z.critic_z_path)



In [19]:
##################################
############## MAIN ##############
##################################

# The dataset should have a column name as signal containing the signals and a column with name anomaly containing the true labels (used during validation)


import json
from sklearn import preprocessing
base_folder = "./Time-Series-Anomaly-Detection-An-experimental-survey/NAB/"
file_name = "artificialWithAnomaly/art_daily_flatmiddle.csv"
label_file = "combined_windows.json"
with open(base_folder + label_file) as FI:
    j_label = json.load(FI)

dataset = pd.read_csv(base_folder + file_name)

dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])


anomalies_span = j_label[file_name]
y = np.zeros(len(dataset))
for w in anomalies_span:
  start = pd.to_datetime(w[0])
  end = pd.to_datetime(w[1])
  for idx in dataset.index:
      if dataset.loc[idx, 'timestamp'] >= start and dataset.loc[idx, 'timestamp'] <= end:
          y[idx] = 1.0
dataset['anomaly'] = y
dataset = dataset.reset_index().drop(columns=['timestamp', 'index'])
dataset.columns = ['signal', 'anomaly']
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
dataset['signal'] = scaler.fit_transform(np.array(dataset['signal']).reshape(-1,1))


#Splitting intro train and test
#TODO could be done in a more pythonic way
train_len = int(0.7 * dataset.shape[0])
dataset[0:train_len].to_csv('train_dataset.csv', index=False)
dataset[train_len:].to_csv('test_dataset.csv', index=False)

train_dataset = SignalDataset(path='train_dataset.csv')
test_dataset = SignalDataset(path='test_dataset.csv')
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True)

logging.info('Number of train datapoints is {}'.format(len(train_dataset)))
logging.info('Number of samples in train dataset {}'.format(len(train_dataset)))

lr = 1e-6

signal_shape = 100
latent_space_dim = 20
encoder_path = 'models/encoder.pt'
decoder_path = 'models/decoder.pt'
critic_x_path = 'models/critic_x.pt'
critic_z_path = 'models/critic_z.pt'

encoder = model.Encoder(encoder_path, signal_shape)
decoder = model.Decoder(decoder_path, signal_shape)
critic_x = model.CriticX(critic_x_path, signal_shape)
critic_z = model.CriticZ(critic_z_path)

mse_loss = torch.nn.MSELoss()

optim_enc = optim.Adam(encoder.parameters(), lr=lr, betas=(0.5, 0.999))
optim_dec = optim.Adam(decoder.parameters(), lr=lr, betas=(0.5, 0.999))
optim_cx = optim.Adam(critic_x.parameters(), lr=lr, betas=(0.5, 0.999))
optim_cz = optim.Adam(critic_z.parameters(), lr=lr, betas=(0.5, 0.999))

train(n_epochs=2000)

anomaly_detection.test(test_loader, encoder, decoder, critic_x)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


Starting training
Epoch 0
Critic training done in epoch 0
Encoder decoder training done in epoch 0
critic x loss 0.055 critic z loss -0.005 
encoder loss 0.307 decoder loss 0.394

Epoch 1
Critic training done in epoch 1
Encoder decoder training done in epoch 1
critic x loss 0.043 critic z loss -0.008 
encoder loss 0.319 decoder loss 0.396

Epoch 2
Critic training done in epoch 2
Encoder decoder training done in epoch 2
critic x loss 0.030 critic z loss -0.012 
encoder loss 0.331 decoder loss 0.401

Epoch 3
Critic training done in epoch 3
Encoder decoder training done in epoch 3
critic x loss 0.018 critic z loss -0.012 
encoder loss 0.343 decoder loss 0.404

Epoch 4
Critic training done in epoch 4
Encoder decoder training done in epoch 4
critic x loss 0.006 critic z loss -0.015 
encoder loss 0.355 decoder loss 0.406

Epoch 5
Critic training done in epoch 5
Encoder decoder training done in epoch 5
critic x loss -0.006 critic z loss -0.018 
encoder loss 0.367 decoder loss 0.403

Epoch 6
C

ZeroDivisionError: ignored